In [1]:
import matplotlib.pyplot as plt

In [2]:
import geopandas as gpd

In [9]:
points = gpd.read_file("geospatialmaps/Landslide Point.shp")
polygons = gpd.read_file("geospatialmaps/Landslide Polygon.shp")
district = gpd.read_file("Tehri_bound/tehri_boundary.shp")

In [10]:
print(points.head())      # see attributes
print(points.shape)       # number of landslides
print(polygons.head())    # polygon attributes
print(polygons.shape)     

   OBJECTID  LONGITUDE   LATITUDE               SLIDE_NO  DEPTH  \
0      2376  78.995028  30.629806  UK/RUD/53J14/2015/188    2.0   
1      2386  78.989667  30.629806  UK/RUD/53J14/2015/208    2.0   
2      2377  78.995100  30.629972  UK/RUD/53J14/2015/189    2.0   
3      2413  78.987806  30.631611  UK/RUD/53J14/2015/201    1.0   
4      2378  78.996361  30.633361  UK/RUD/53J14/2015/190    2.0   

            GEOLOGY    STRUCTURE ABSTRACT  \
0  Quartzite gneiss  Low jointed     None   
1  Quartzite gneiss         None     None   
2  Quartzite gneiss  Low jointed     None   
3  Quartzite gneiss         None     None   
4            Gneiss      sheared     None   

                                            CITATION  LENGTH  ...  \
0  Vinay et al., 2016. Macro-scale (1:50,000) lan...    70.0  ...   
1  Vinay et al., 2016. Macro-scale (1:50,000) lan...    80.0  ...   
2  Vinay et al., 2016. Macro-scale (1:50,000) lan...    30.0  ...   
3  Vinay et al., 2016. Macro-scale (1:50,000) lan.

In [11]:
# --- Ensure same CRS (lat/lon WGS84) ---
points = points.to_crs(epsg=4326)
polygons = polygons.to_crs(epsg=4326)
district = district.to_crs(epsg=4326)

In [7]:
import folium

In [12]:
# --- Map center ---
center = [district.geometry.centroid.y.mean(), district.geometry.centroid.x.mean()]
m = folium.Map(location=center, zoom_start=9, tiles="OpenStreetMap")

C:\Users\sangh\AppData\Local\Temp\ipykernel_20756\2726828719.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = [district.geometry.centroid.y.mean(), district.geometry.centroid.x.mean()]


In [13]:
# --- Add district boundary ---
folium.GeoJson(
    district,
    name="Tehri Garhwal Boundary",
    style_function=lambda x: {"color": "black", "weight": 2, "fillOpacity": 0}
).add_to(m)

In [14]:
# --- Add polygons (landslide areas) ---
folium.GeoJson(
    polygons,
    name="Landslide Polygons",
    style_function=lambda x: {"color": "red", "weight": 1, "fillOpacity": 0.3}
).add_to(m)

In [15]:
# --- Add points (landslide locations) ---
for _, row in points.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=3,
        color="blue",
        fill=True,
        fill_opacity=0.7
    ).add_to(m)

In [16]:
# --- Controls & save ---
folium.LayerControl().add_to(m)
m.save("tehri_landslides_map.html")
